# Variable weights derivation

In [1]:
import copy
import numpy as np
import pandas as pd
import xarray as xr

## Tropopause weights

In [2]:
N_levels = 18
base_dir = '/glade/derecho/scratch/ksha/CREDIT_data/ERA5_mlevel_1deg/'
ds_example = xr.open_zarr(base_dir+'all_in_one/ERA5_mlevel_1deg_6h_subset_1979_conserve.zarr')
mlevel_sub = ds_example['level'].values

In [3]:
ds_static = xr.open_zarr(base_dir+'static/ERA5_mlevel_1deg_static_subset.zarr')
coef_a = ds_static['coef_a'].values
coef_b = ds_static['coef_b'].values

In [4]:
plevel_sub_bounds = (coef_a + coef_b*101300)/100

In [5]:
plevel_sub = 0.5*(plevel_sub_bounds[1:] + plevel_sub_bounds[:-1])

In [6]:
plevel_sub[9]

322.2688289985806

In [7]:
W_18 = plevel_sub/plevel_sub.sum()
W_18[10:] = W_18[-1]
W_18[9] = 0.5*(W_18[8]+W_18[10])
print(W_18)

[1.36577589e-06 2.38872094e-05 2.06553177e-04 8.92194675e-04
 2.53615808e-03 5.55109728e-03 1.02659148e-02 1.74081408e-02
 2.82033917e-02 8.25636383e-02 1.36923885e-01 1.36923885e-01
 1.36923885e-01 1.36923885e-01 1.36923885e-01 1.36923885e-01
 1.36923885e-01 1.36923885e-01]


In [9]:
W_18_q = W_18.copy()
W_18_q[1:] = 0.5*W_18_q[1:]

In [1]:
#3*W_18.sum() + W_18_q.sum() + 0.058*4 + 0.029*9

In [12]:
base_dir = '/glade/derecho/scratch/ksha/CREDIT_data/ERA5_mlevel_1deg/'
ds_residual = xr.open_dataset(base_dir+'mean_std/residual_original_6h_1979_2019_conserve_1deg.nc')

varnames = list(ds_residual.keys())
varnames_upper_air = ['specific_total_water', 'temperature', 'u_component_of_wind','v_component_of_wind']
varnames_surf = ['SP', 'VAR_2T', 'VAR_10U', 'VAR_10V']
varnames_diag = list(set(varnames) - set(varnames_upper_air) - set(varnames_surf))

In [16]:
for var in varnames_upper_air:
    if var == 'specific_total_water':
        W_18_q = W_18.copy()
        W_18_q[1:] = 0.5*W_18_q[1:]
        print('{}: {}'.format(var, W_18_q))
    else:
        print('{}: {}'.format(var, W_18))

for var in varnames_surf:
    print('{}: {}'.format(var, 0.13))

for var in varnames_diag:
    print('{}: {}'.format(var, 0.065))

specific_total_water: [1.36577589e-06 1.19436047e-05 1.03276589e-04 4.46097338e-04
 1.26807904e-03 2.77554864e-03 5.13295742e-03 8.70407041e-03
 1.41016958e-02 4.12818192e-02 6.84619425e-02 6.84619425e-02
 6.84619425e-02 6.84619425e-02 6.84619425e-02 6.84619425e-02
 6.84619425e-02 6.84619425e-02]
temperature: [1.36577589e-06 2.38872094e-05 2.06553177e-04 8.92194675e-04
 2.53615808e-03 5.55109728e-03 1.02659148e-02 1.74081408e-02
 2.82033917e-02 8.25636383e-02 1.36923885e-01 1.36923885e-01
 1.36923885e-01 1.36923885e-01 1.36923885e-01 1.36923885e-01
 1.36923885e-01 1.36923885e-01]
u_component_of_wind: [1.36577589e-06 2.38872094e-05 2.06553177e-04 8.92194675e-04
 2.53615808e-03 5.55109728e-03 1.02659148e-02 1.74081408e-02
 2.82033917e-02 8.25636383e-02 1.36923885e-01 1.36923885e-01
 1.36923885e-01 1.36923885e-01 1.36923885e-01 1.36923885e-01
 1.36923885e-01 1.36923885e-01]
v_component_of_wind: [1.36577589e-06 2.38872094e-05 2.06553177e-04 8.92194675e-04
 2.53615808e-03 5.55109728e-03 1.0

```python
    variable_weights:
        specific_total_water: [1.36577589e-06, 1.19436047e-05, 1.03276589e-04, 4.46097338e-04, 1.26807904e-03, 2.77554864e-03, 5.13295742e-03, 8.70407041e-03, 1.41016958e-02, 4.12818192e-02, 6.84619425e-02, 6.84619425e-02, 6.84619425e-02, 6.84619425e-02, 6.84619425e-02, 6.84619425e-02, 6.84619425e-02, 6.84619425e-02]
        temperature: [1.36577589e-06, 2.38872094e-05, 2.06553177e-04, 8.92194675e-04, 2.53615808e-03, 5.55109728e-03, 1.02659148e-02, 1.74081408e-02, 2.82033917e-02, 8.25636383e-02, 1.36923885e-01, 1.36923885e-01, 1.36923885e-01, 1.36923885e-01, 1.36923885e-01, 1.36923885e-01, 1.36923885e-01, 1.36923885e-01]
        u_component_of_wind: [1.36577589e-06, 2.38872094e-05, 2.06553177e-04, 8.92194675e-04, 2.53615808e-03, 5.55109728e-03, 1.02659148e-02, 1.74081408e-02, 2.82033917e-02, 8.25636383e-02, 1.36923885e-01, 1.36923885e-01, 1.36923885e-01, 1.36923885e-01, 1.36923885e-01, 1.36923885e-01, 1.36923885e-01, 1.36923885e-01]
        v_component_of_wind: [1.36577589e-06, 2.38872094e-05, 2.06553177e-04, 8.92194675e-04, 2.53615808e-03, 5.55109728e-03, 1.02659148e-02, 1.74081408e-02, 2.82033917e-02, 8.25636383e-02, 1.36923885e-01, 1.36923885e-01, 1.36923885e-01, 1.36923885e-01, 1.36923885e-01, 1.36923885e-01, 1.36923885e-01, 1.36923885e-01]
        SP: 0.13
        VAR_2T: 0.13
        VAR_10U: 0.13
        VAR_10V: 0.13
        surface_latent_heat_flux: 0.065
        surface_net_solar_radiation: 0.065
        evaporation: 0.065
        surface_net_thermal_radiation: 0.065
        toa_incident_solar_radiation: 0.065
        surface_sensible_heat_flux: 0.065
        total_precipitation: 0.065
        top_net_thermal_radiation: 0.065
        top_net_solar_radiation: 0.065
```